In [1]:
import os
import sys
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import spacy

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config,  AutoTokenizer, AutoModelWithLMHead

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [3]:
!python3 --version

Python 3.8.8


In [5]:
train = pd.read_csv('~/qg_dataset_supporting_facts/hotpot_train_v1.1.csv')
train = train[:10000]
valid = pd.read_csv('~/qg_dataset_supporting_facts/hotpot_dev_distractor_v1.csv')
valid = valid[:1000]

train.head()

,Unnamed: 0,question,text
0,5a7a06935542990198eaf050,Which magazine was started first Arthur's Maga...,<answer> Arthur's Magazine <context> Arthur's ...
1,5a879ab05542996e4f30887e,The Oberoi family is part of a hotel company t...,<answer> Delhi <context> The Oberoi family is ...
2,5a8d7341554299441c6b9fe5,Musician and satirist Allie Goertz wrote a son...,<answer> President Richard Nixon <context> All...
3,5a82171f5542990a1d231f4a,What nationality was James Henry Miller's wife?,<answer> American <context> Margaret Peggy See...
4,5a84dd955542997b5ce3ff79,Cadmium Chloride is slightly soluble in this c...,<answer> alcohol <context> It is a hygroscopi...


In [8]:
PRETRAINED_MODEL = 't5-base'
DIR = "question_generator/"
BATCH_SIZE = 1
SEQ_LENGTH = 512

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

tokenizer.add_special_tokens(
    {'additional_special_tokens': ['<answer>', '<context>']}
)
class QGDataset(Dataset):
    def __init__(self, csv):
        self.df = csv

    def __len__(self):
         return len(self.df)

    def __getitem__(self, idx):   
        if torch.is_tensor(idx):
            idx = idx.tolist()
        row = self.df.iloc[idx, 1:]       

        encoded_text = tokenizer(
            row['text'], 
            padding=True, 
            max_length=SEQ_LENGTH,
            truncation=True,
            return_tensors="pt"
        )
        encoded_text['input_ids'] = torch.squeeze(encoded_text['input_ids'])
        encoded_text['attention_mask'] = torch.squeeze(encoded_text['attention_mask'])

        encoded_question = tokenizer(
            row['question'],
            padding=True,
            max_length=SEQ_LENGTH,
            truncation=True,
            return_tensors='pt'
        )
        encoded_question['input_ids'] = torch.squeeze(encoded_question['input_ids'])

        return (encoded_text.to(device), encoded_question.to(device))

train_set = QGDataset(train)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
valid_set = QGDataset(valid)
valid_loader = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=False)

In [58]:
LR = 0.001
EPOCHS = 20
LOG_INTERVAL = 5000

config = T5Config(decoder_start_token_id=tokenizer.pad_token_id)
model = T5ForConditionalGeneration(config).from_pretrained(PRETRAINED_MODEL)
model.resize_token_embeddings(len(tokenizer)) # to account for new special tokens
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

In [59]:
SAVED_MODEL_PATH = "/home2/samyak.ja/qg_dataset/model_hotpot.pth"
TEMP_SAVE_PATH = "/home2/samyak.ja/qg_dataset/model_hotpot.pth"

def train(epoch, best_val_loss):
    model.train()
    total_loss = 0.
    for batch_index, batch in enumerate(train_loader):
        data, target = batch
        optimizer.zero_grad()
        masked_labels = mask_label_padding(target['input_ids'])
        output = model(
            input_ids=data['input_ids'],
            attention_mask=data['attention_mask'],
            labels=masked_labels
        )
        loss = output[0]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch_index % LOG_INTERVAL == 0 and batch_index > 0:
            cur_loss = total_loss / LOG_INTERVAL
            print('| epoch {:3d} | ' 
                  '{:5d}/{:5d} batches | '
                  'loss {:5.2f}'.format(
                    epoch, 
                    batch_index, len(train_loader), 
                    cur_loss))
            save(
                TEMP_SAVE_PATH,
                epoch, 
                model.state_dict(), 
                optimizer.state_dict(), 
                best_val_loss
            )
            total_loss = 0

def evaluate(eval_model, data_loader):
    eval_model.eval()
    total_loss = 0.
    with torch.no_grad():
        for batch_index, batch in enumerate(data_loader):
            data, target = batch
            masked_labels = mask_label_padding(target['input_ids'])
            output = eval_model(
                input_ids=data['input_ids'],
                attention_mask=data['attention_mask'],
                labels=masked_labels
            )
            total_loss += output[0].item()
    return total_loss / len(data_loader)

def mask_label_padding(labels):
    MASK_ID = -100
    labels[labels==tokenizer.pad_token_id] = MASK_ID
    return labels

def save(path, epoch, model_state_dict, optimizer_state_dict, loss):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model_state_dict,
            'optimizer_state_dict': optimizer_state_dict,
            'best_loss': loss,
            }, path)

def load(path):
    return torch.load(path)

def print_line():
    LINE_WIDTH = 60
    print('-' * LINE_WIDTH)

In [60]:
best_val_loss = float("inf")
best_model = None

val_loss = evaluate(model, valid_loader)
print_line()
print('| Before training | valid loss {:5.2f}'.format(
    val_loss)
)
print_line()

------------------------------------------------------------
| Before training | valid loss  4.00
------------------------------------------------------------


In [61]:
for epoch in range(1, EPOCHS + 1):

    train(epoch, best_val_loss)
    val_loss = evaluate(model, valid_loader)
    print_line()
    print('| end of epoch {:3d} | valid loss {:5.2f}'.format(
        epoch,
        val_loss)
    )
    print_line()

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model
        save(
             SAVED_MODEL_PATH,
             epoch, 
             model.state_dict(), 
             optimizer.state_dict(), 
             best_val_loss
        )
        print("| Model saved.")
        print_line()

| epoch   1 |  5000/10000 batches | loss  3.21
------------------------------------------------------------
| end of epoch   1 | valid loss  2.72
------------------------------------------------------------
| Model saved.
------------------------------------------------------------
| epoch   2 |  5000/10000 batches | loss  2.83
------------------------------------------------------------
| end of epoch   2 | valid loss  2.55
------------------------------------------------------------
| Model saved.
------------------------------------------------------------
| epoch   3 |  5000/10000 batches | loss  2.67
------------------------------------------------------------
| end of epoch   3 | valid loss  2.43
------------------------------------------------------------
| Model saved.
------------------------------------------------------------
| epoch   4 |  5000/10000 batches | loss  2.58
------------------------------------------------------------
| end of epoch   4 | valid loss  2.33
-----

In [62]:
torch.save(model, "/home2/samyak.ja/qg_dataset/model_hotpot_last.pth")

In [9]:
md2 = torch.load("/home2/samyak.ja/qg_dataset_supporting_facts/model_hotpot_last.pth")

In [64]:
val_loss = evaluate(md2, valid_loader)

In [65]:
print('| Before training | valid loss {:5.2f}'.format(
    val_loss)
)

| Before training | valid loss  1.92


In [10]:
def inference(review_text, model, device):
    encoded_text = tokenizer(
            review_text, 
            padding=True, 
            max_length=SEQ_LENGTH,
            truncation=True,
            return_tensors="pt"
        ).to(device)

    input_ids = encoded_text['input_ids']
    with torch.no_grad():
        output = model.generate(input_ids)
    decoded_string = tokenizer.decode(output[0], skip_special_tokens=True)
    print(decoded_string)

In [11]:
review_text = "<answer> a fusional language <context> Typologically, Estonian represents a transitional form from an agglutinating language to a fusional language. The canonical word order is SVO (subject–verb–object)."

In [12]:
inference(review_text, md2, device)

Estonian is a transitional form from an agglutinating language to what
